In [2]:
!pip install bitstring

  Obtaining dependency information for bitstring from https://files.pythonhosted.org/packages/59/bc/bbc41ad3546f23855a2c21dc6afcd8b148ccec2e51a5af145199abfa4b9e/bitstring-4.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for bitarray<3.0.0,>=2.8.0 from https://files.pythonhosted.org/packages/24/2a/e601104038cde9537a3044f6ec824bb9237f763579bc361138824825fb3c/bitarray-2.8.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
   ---------------------------------------- 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/123.0 kB ? eta -:--:--
   ----------------------------- ---------- 92.2/123.0 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 123.0/123.0 kB 1.8 MB/s eta 0:00:00


In [4]:
from bitstring import BitArray

# SBox for encryption
sbox_enc = [0xa, 0x0, 0x9, 0xe, 0x6, 0x3, 0xf, 0x5,
            0x1, 0xd, 0xc, 0x7, 0xb, 0x4, 0x2, 0x8]

# Inverse SBox
sbox_dec = [0xa, 0x5, 0xe, 0xf, 0x8, 0xc, 0x1, 0x2,
            0xd, 0xb, 0x4, 0x6, 0x3, 0x0, 0x7, 0x9]

# Implement ShiftRow
def shift_row(block):
    shifted = BitArray(16)
    shifted[0:4] = block[4:8]
    shifted[4:8] = block[0:4]
    shifted[8:12] = block[8:12]
    shifted[12:16] = block[12:16]
    return shifted

# MixColumns matrix
mc_mat = [BitArray(hex='02'), BitArray(hex='03'),
          BitArray(hex='01'), BitArray(hex='01')]

# Implement MixColumns
def mix_columns(block):
    mixed = BitArray(16)
    for i in range(4):
        col = BitArray(4)
        for j in range(4):
            col[j] = block[(i + 4*j) % 16]
        for j in range(4):
            mixed[(i + 4*j) % 16] = mc_mat[j] & col
    return mixed

# Inverse MixColumns matrix
inv_mc_mat = [BitArray(hex='0e'), BitArray(hex='09'),
              BitArray(hex='0d'), BitArray(hex='0b')]

# Implement Inverse MixColumns
def inv_mix_columns(block):
    mixed = BitArray(16)
    for i in range(4):
        col = BitArray(4)
        for j in range(4):
            col[j] = block[(i + 4*j) % 16]
        for j in range(4):
            mixed[(i + 4*j) % 16] = inv_mc_mat[j] & col
    return mixed

# Implement SubNibbles
def sub_nibbles(block, sbox):
    subbed = BitArray(16)
    for i in range(4):
        nibble = BitArray(4)
        for j in range(4):
            nibble[j] = block[(i*4 + j)]
        subbed[(i*4):(i*4+4)] = BitArray(uint=sbox[int(nibble.hex, 16)], length=4)
    return subbed

# Implement key schedule
rcon1 = BitArray(hex='0x01')
rcon2 = BitArray(hex='0x02')

def generate_round_keys(master_key):
    k1 = master_key.copy()
    k2 = BitArray(16)

    # Generate k1
    k1[0:4] = k1[0:4] ^ BitArray(uint=sbox_enc[k1[12:16].uint], length=4) ^ rcon1
    k1[4:8] = k1[4:8] ^ k1[0:4]
    k1[8:12] = k1[8:12] ^ k1[4:8]
    k1[12:16] = k1[12:16] ^ k1[8:12]

    # Generate k2
    k2[0:4] = k1[0:4] ^ BitArray(uint=sbox_enc[k1[12:16].uint], length=4) ^ rcon2
    k2[4:8] = k1[4:8] ^ k2[0:4]
    k2[8:12] = k1[8:12] ^ k2[4:8]
    k2[12:16] = k1[12:16] ^ k2[8:12]

    return k1, k2

# Encrypt
def encrypt_block(plaintext, key):
    k1, k2 = generate_round_keys(key)

    plaintext = sub_nibbles(plaintext, sbox_enc)
    plaintext = shift_row(plaintext)
    plaintext = mix_columns(plaintext)
    plaintext = plaintext ^ k1
    plaintext = sub_nibbles(plaintext, sbox_enc)
    plaintext = shift_row(plaintext)
    ciphertext = plaintext ^ k2

    return ciphertext


# Decrypt

def decrypt_block(ciphertext, key):

    k1, k2 = generate_round_keys(key)

    plaintext = ciphertext ^ k2
    plaintext = shift_row(plaintext)
    plaintext = sub_nibbles(plaintext, sbox_dec)
    plaintext = plaintext ^ k1
    plaintext = inv_mix_columns(plaintext)
    plaintext = shift_row(plaintext)
    plaintext = sub_nibbles(plaintext, sbox_dec)

    return plaintext

def main():
    # Get user input for the text block and key
    text_block_hex = input("Enter a text block in hexadecimal format: ").strip()
    key_hex = input("Enter a key in hexadecimal format: ").strip()

    # Convert hexadecimal input to BitArray
    text_block = BitArray(hex=text_block_hex)
    key = BitArray(hex=key_hex)

    # Perform the encryption operations
    ciphertext = encrypt_block(text_block, key)

    # Display the results
    print(f"SubNibbles({text_block_hex}) = {sub_nibbles(text_block, sbox_enc).hex}")
    print(f"ShiftRow({text_block_hex}) = {shift_row(text_block).hex}")
    print(f"MixColumns({text_block_hex}) = {mix_columns(text_block).hex}")
    print(f"GenerateRoundKeys({key_hex}) = ({generate_round_keys(key)[0].hex}, {generate_round_keys(key)[1].hex})")
    print(f"Encrypted Block: {ciphertext.hex}")

if __name__ == "__main__":
    main()

SubNibbles(903b) = dae7
ShiftRow(903b) = 309b
MixColumns(903b) = 9297
GenerateRoundKeys(02cc) = ('57b7', 'ad61')


# Part 3

In [ ]:
with open('secret.txt', 'r') as file:
    content = file.read()
print(content)

content = content.split()
plain = []
for i in range(len(content)):
    plain.append(D2(content[i], '149c'))
final = []
for i in range(len(plain)):
    final.append(chr(int(plain[i][:2],16)))
    final.append(chr(int(plain[i][2:],16)))
final = "".join(final)
final[:-2]